In [1]:
import pandas

In [2]:
lines = None
with open("data/features", 'r') as stream:
    lines = map(lambda x: x.strip(), stream.readlines())

In [3]:
repro = pandas.read_csv("data/repro.csv")

In [4]:
table = dict()
for elt in lines: 
    table[elt] = [None for _ in range(1000)]
table["BUILD?"] = list(repro["build?"])
table["REPRO?"] = list(repro["reproducible?"])

In [5]:
for i in range(1, 1001):
    config = f"configs/{i:04d}_randconfig"
    with open(config, 'r') as stream:
        for line in stream:
            if line.startswith('#'):
                continue
            if line.startswith("CONFIG_"):
                name, value = line.strip().split('=')
                name = name[7:]
                table[name][i-1] = value

In [6]:
df = pandas.DataFrame.from_dict(table)

In [7]:
df

,104_QUAD_8,60XX_WDT,64BIT,6LOWPAN,6LOWPAN_DEBUGFS,6LOWPAN_GHC_EXT_HDR_DEST,6LOWPAN_GHC_EXT_HDR_FRAG,6LOWPAN_GHC_EXT_HDR_HOP,6LOWPAN_GHC_EXT_HDR_ROUTE,6LOWPAN_GHC_ICMPV6,...,ZSTD_COMPRESS,ZSTD_DECOMPRESS,ZSWAP,ZSWAP_COMPRESSOR_DEFAULT,ZSWAP_COMPRESSOR_DEFAULT_LZO,ZSWAP_DEFAULT_ON,ZSWAP_ZPOOL_DEFAULT,ZSWAP_ZPOOL_DEFAULT_ZBUD,BUILD?,REPRO?
0,None,None,y,y,None,None,None,None,None,None,...,y,y,None,None,None,None,None,None,True,True
1,None,y,y,None,None,None,None,None,None,None,...,None,m,None,None,None,None,None,None,True,True
2,None,y,y,None,None,None,None,None,None,None,...,y,y,None,None,None,None,None,None,True,False
3,None,y,y,None,None,None,None,None,None,None,...,y,y,None,None,None,None,None,None,True,True
4,None,None,y,None,None,None,None,None,None,None,...,None,y,None,None,None,None,None,None,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,None,None,y,None,None,None,None,None,None,None,...,m,y,None,None,None,None,None,None,True,False
996,None,y,y,None,None,None,None,None,None,None,...,y,y,None,None,None,None,None,None,True,True
997,None,None,y,None,None,None,None,None,None,None,...,y,y,None,None,None,None,None,None,True,True
998,None,None,y,None,None,None,None,None,None,None,...,y,y,None,None,None,None,None,None,True,True
